In [1]:
from langchain_community.chat_message_histories import ChatMessageHistory
from app.chroma_store import get_chroma_db
from app.agent import build_agent
from langchain_openai import ChatOpenAI
from app.config import LLM_MODEL, OPENAI_API_KEY, OPENAI_API_BASE, TEMPERATURE
from langchain_core.output_parsers import JsonOutputKeyToolsParser
from langchain_core.runnables import RunnablePassthrough
import importlib
import prompts  # the whole folder
importlib.reload(prompts)

from prompts import llm_judge_prompt

In [2]:
store = {}

def get_session_history(session_id: str):
    return store.setdefault(session_id, ChatMessageHistory())

In [3]:
db = get_chroma_db()
retriever = db.as_retriever()
qa_agent = build_agent(retriever, get_session_history)

In [54]:
query = 'Did Ariana Grande marry pete?'
session_id = '1'

In [55]:
response = qa_agent.invoke(
            {"input": query},
            config={"configurable": {"session_id": session_id}}
        )

print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `get_band_information` with `{'query': 'Ariana Grande'}`


{'context': 'Influences\n\nGenres\n\nFor the book Under My Thumb: Songs That Hate Women and the Women Who Love Them, The Guardian arts critic Fiona Sturges contributed an essay evaluating her love for AC/DC. While acknowledging she is a feminist and that the band\'s music is problematic for her, she believed it would be "daft as opposed to damaging" for female listeners if they could understand the band to be "a bunch of archly sex-obsessed idiots with sharp tunes and some seriously killer riffs". Despite the "unpleasant sneering quality" of "Carry Me Home"\'s claims about a woman who "ain\'t no lady", the "rape fantasy" of "Let Me Put My Love into You" and the generally one-dimensional portrayals of women, Sturges said songs such as "Whole Lotta Rosie" and "You Shook Me All Night Long" demonstrated that the female characters "are also having a good time and are, more often tha

In [56]:
def format_tool_calls(intermediate_steps):
    tool_calls = []
    for step, _ in intermediate_steps:
        if hasattr(step, 'tool') and hasattr(step, 'tool_input'):
            tool_calls.append(f"{step.tool} called with {step.tool_input}")
    return "\n".join(tool_calls)

tool_calls_summary = format_tool_calls(response['intermediate_steps'])

print(tool_calls_summary)

get_band_information called with {'query': 'Ariana Grande'}


In [57]:
judge_llm = ChatOpenAI(
        model=LLM_MODEL,
        api_key=OPENAI_API_KEY,
        base_url=OPENAI_API_BASE,
        temperature=TEMPERATURE
    )

judge_chain = llm_judge_prompt | judge_llm 

In [58]:
judge_response = judge_chain.invoke({
    "query": query,
    "final_answer": response['output'],
    "tool_calls": tool_calls_summary,
    "chat_history": store[session_id].messages
})


In [59]:
import json

raw_content = judge_response.content

# Clean it: remove ```json and ``` if present
cleaned_content = raw_content.strip()
if cleaned_content.startswith("```json"):
    cleaned_content = cleaned_content[len("```json"):].strip()
if cleaned_content.startswith("```"):
    cleaned_content = cleaned_content[len("```"):].strip()
if cleaned_content.endswith("```"):
    cleaned_content = cleaned_content[:-3].strip()

result = json.loads(cleaned_content)
print(result['relevant'])

True
